# Ecological Sensitvity Clean up and Initial Analysis
The purpose of this notebook is to clean the data and create a table with the average "seagrass-association" score for each target species. Individual scores were provided by local key informants in each community.

In [22]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from zipfile import ZipFile
import os

In [23]:
# unzipping survey tables

# specifying location of .zip and extraction location
sur_A_path="/home/rthoms/ocean_dep/sur_A/raw_data/Sur_A_26072023.zip"
sur_A_raw="/home/rthoms/ocean_dep/sur_A/raw_data/sur_A_raw"

# loading the .zip and creating a zip object
with ZipFile(sur_A_path, 'r') as zObject:

	# Extracting all the members of the zip
	# into a the specified location.
	zObject.extractall(
		path=sur_A_raw)

The survey folder contains a parent table ("Ecological Vulnerability Data Entry_0"), where each row is the responses of one key informant, and a child table ("resource_table_1"), where each row is a resource and is associated with a response in the parent table.

In [24]:
# read in the data tables
res_raw = os.path.join(sur_A_raw, "resource_table_1.csv")
df_res = pd.read_csv(res_raw)

ev_raw = os.path.join(sur_A_raw, "Ecological Vulnerability Data Entry_0.csv")
df_ev = pd.read_csv(ev_raw)
df_ev = df_ev.rename(columns={'globalid': 'ParentGlobalID'})
df_ev

,objectid,ParentGlobalID,id,date,ccp,seagrass_quality_change,sg_quality_change_note,seagrass_quantity_change,sg_quantity_change_note,causes_seagrass_change,...,resource_change_note,resource_change_note.1,note,review_flag,CreationDate,Creator,EditDate,Editor,x,y
0,2,b6f6afb4-7c5a-44a1-97d5-74482f766527,05032023JOSA01,5/8/2023 10:00:00 AM,josina_machel,sig_worse,Ervas secaram e sairam. O tamanho reduziu_se ...,sig_less,"Afecta pirony, gywindrituno, kelegele perderam...",As mudancas climaticas como as temperaturas al...,...,Leia 1.C,"human_disturbance,climate_change",\n\n ...,0.0,5/17/2023 11:08:09 AM,rthoms_wri,5/17/2023 11:08:09 AM,rthoms_wri,0,0
1,3,2f7b61dd-e0c8-47f0-9c0b-9879a3cee5a1,04022023CHAA01,5/8/2023 10:00:00 AM,chamane,no_change,"Nao ha mudanca, as eravas ce antiguidade eram ...",sig_less,Ve_se muita reducao das ervas marinhas por cau...,"As redes de arrasto sao o ,maior factor dessa ...",...,"A qualidade assim como a quantidade , reduzira...",human_disturbance,\n\n ...,0.0,5/17/2023 11:08:10 AM,rthoms_wri,5/17/2023 11:08:10 AM,rthoms_wri,0,0
2,4,4936c582-ca79-48b3-8919-e4ab7dbb035d,18022023ILHA01,5/20/2023 10:00:00 AM,ilha,sig_worse,"O vendaval de 2017, fez com que houvesse a dev...",sig_less,Leia 1a.,"Os ciclones e as redes de arrasto, sao po maio...",...,"Para as especies que notou_se a mudanca, leia ...",human_disturbance,\n\n ...,0.0,5/20/2023 11:28:17 AM,rthoms_wri,5/20/2023 11:28:17 AM,rthoms_wri,0,0
3,5,4ea76d43-3ff2-4b74-afb7-795019a4b197,02022023MARA01,5/20/2023 10:00:00 AM,marrambone,sig_worse,Agora devido as artes que se introduzem na bai...,sig_less,As ervas diminuiram mas ve_se melhoria porque ...,O que fez com que houvesse a mudanca foi a in...,...,Ha dois factores que contribuem nessas mudanca...,"human_disturbance,climate_change",\n\n ...,0.0,5/20/2023 11:28:19 AM,rthoms_wri,5/20/2023 11:28:19 AM,rthoms_wri,0,0
4,6,d298ff03-a3ac-4257-90e0-a10890bf0494,31012023MUCA01,5/20/2023 10:00:00 AM,mucucune,no_change,"A partir da decada 70 a 80, havia muita erva m...",less,"Haviamos perdido as ervas, a pertir do ano 201...",ON que causpou esta mudanca foi a introducao d...,...,"O uso da pesca, das artes e da forma de pescar...",human_disturbance,\n\n ...,0.0,5/20/2023 12:07:47 PM,rthoms_wri,5/20/2023 12:07:47 PM,rthoms_wri,0,0
5,7,e7664a71-64d4-4f13-ad4b-61b1ba63b82d,03282023NHMUA01,3/28/2023 10:00:00 AM,nhamua,sig_worse,Houve se mudanca porque antes so pescaram em c...,sig_less,Houve se grande persa das ervas marinhas. Agor...,Antigamenete redes nao tinham ferros que arras...,...,Redes mosquiteros e redes nocivas capturam pei...,human_disturbance,NaN,NaN,5/23/2023 2:35:21 PM,rthoms_wri,5/23/2023 2:37:01 PM,rthoms_wri,0,0
6,8,3b6c93d5-8c88-4ae9-8150-5c66e95d9bc6,04032023NGUA01,5/24/2023 10:00:00 AM,nguja,sig_worse,"Na entrada da baia de Tumbine piorou, desde da...",sig_less,"Acabou, muita quantidade se foi.",As proprias pessoas que usam jantes e pneus de...,...,A quantidade baixou por causa de todas pessoas...,"human_disturbance,climate_change",\n\n ...,0.0,5/24/2023 11:29:28 AM,rthoms_wri,5/24/2023 11:29:28 AM,rthoms_wri,0,0
7,9,3bfefeff-a922-4545-b451-ac01818ea943,17022023NHCHA01,5/24/2023 10:00:00 AM,nhamaxaxa,sig_worse,NaN,sig_less,"Na altura, existiam canais que tinham ervas ma...",Leia 1b.,...,A forma de pescar nos ultimos dias e diferente...,human_disturbance,This was entered as 65 but should be 71. Edite...,0.0,5/24/2023 11:29:29 AM,rthoms_wri,7/26/2023 7:48:49 PM,rthoms_wri,0,0
8,10,37c82799-2da6-46b1-b248-26516978bbba,10032023SAHA01,5/24/2023 10:00:00 AM,sahane,sig_worse,NaN,sig_less,NaN,O que causa as mudancas na qualidade assim com...,...,Leia 1c,"climate_change,human_disturbance",\n\n ...,0.0,5/24/2023 11:29:32 AM,rthoms_wri,5/24/2023 11:29:32 AM,rthoms_wri,0,0
9,11,46efa4d9-8fb1-41e1-843c-34ebaf1db021,09022023NHMPA01,5/24/2023 10:00:00 AM,nhampossa,worse,A qualidade das ervas marinhas esta pior por c...,less,Ve_se mudanca devido a estas redes de arrasto ...,Leia 1a e 1b,...,O que faz com que haja 

In [25]:
# unfortuneatley only the resource labels were stored in the survey responses in the child table
# the code below creates a regex to isolate the ID numbers from each resource label

# regex pattern for two consecutive numbers
regex = r'[1-9][1-9]'
df_res['focal_resource']=df_res['_focal_resource_label'].str.replace(r'[^(]*\(|\)[^)]*', '')

# funtion to add a zero to the begining of single digit ids (ie 1 --> 01)
def add_0(id):
    if len(str(id))==1:
        id = '0'+ str(id)
    else:
        id = str(id)
    return id

# create an id column for the focal resource using the above function
df_res['focal_resource']= df_res.apply(lambda row: add_0(row['focal_resource']), axis=1 )
df_res

/tmp/ipykernel_1588/1486822861.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_res['focal_resource']=df_res['_focal_resource_label'].str.replace(r'[^(]*\(|\)[^)]*', '')


,ObjectID,GlobalID,_focal_resource_label,sg_dependence,resource_quality_change,resource_quantity_change,management,compliance,ParentGlobalID,CreationDate,Creator,EditDate,Editor,management_other,focal_resource
0,9,c90b73f9-9d9b-41e5-8a3d-78903a48426b,mihili (1),low,sig_worse,sig_less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,5/17/2023 11:08:09 AM,rthoms_wri,5/17/2023 11:08:09 AM,rthoms_wri,veda de area,01
1,10,137bfef8-be26-49d3-a0bf-3e864b58fb34,mavionho (2),high,sig_worse,less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,5/17/2023 11:08:09 AM,rthoms_wri,5/17/2023 11:08:09 AM,rthoms_wri,Veda de area,02
2,11,95c4a517-3e77-4f10-97f7-82bc3ecc9a33,salani (3),low,sig_worse,sig_less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,5/17/2023 11:08:09 AM,rthoms_wri,5/17/2023 11:08:09 AM,rthoms_wri,Veda de area,03
3,12,b7af52e9-5b57-4bd3-a8ba-7f4186cb7cc7,sigulula (6),low,sig_worse,sig_less,other,75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,5/17/2023 11:08:09 AM,rthoms_wri,7/26/2023 8:31:22 PM,rthoms_wri,veda de area,06
4,13,9e4bdef5-fc9f-4588-ab71-2e4eeb4f0839,dzindrolo (50),high,sig_worse,sig_less,"gear_closure,other",75_100,b6f6afb4-7c5a-44a1-97d5-74482f766527,5/17/2023 11:08:09 AM,rthoms_wri,5/17/2023 11:08:09 AM,rthoms_wri,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,92,7c18cd81-f219-41b6-afd3-703dfa46eba4,dzindrolo (50),mod,worse,sig_less,gear_closure,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,5/25/2023 6:03:24 AM,rthoms_wri,5/25/2023 6:03:24 AM,rthoms_wri,NaN,50
84,93,0dabe7a0-18b3-4f81-a9c2-e098ba5fac57,"gudua, girumba (55)",high,no_change,sig_less,gear_closure,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,5/25/2023 6:03:24 AM,rthoms_wri,7/26/2023 8:34:42 PM,rthoms_wri,NaN,55
85,94,40041e6b-4867-4657-ba6e-05f3b24a6419,mavandzi (56),none,sig_worse,sig_less,gear_closure,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,5/25/2023 6:03:24 AM,rthoms_wri,5/25/2023 6:03:24 AM,rthoms_wri,NaN,56
86,95,ec1633a5-e48e-4129-9e49-21650cb7e0d4,mahutu (64),none,no_change,no_change,other,100,05d15b5d-f3fa-4464-8670-8bab8e40017c,5/25/2023 6:03:24 AM,rthoms_wri,5/25/2023 6:03:24 AM,rthoms_wri,NaN,64


In [36]:
# read in the species table (descriptive information about each species)
species = pd.read_csv("/home/rthoms/ocean_dep/sur_B/clean_up/species.csv")

# rename resource id column and merge with the resource dataframe
species.rename(columns={'resource_id':'focal_resource'}, inplace=True)
df_wk = pd.merge(df_res,species,on='focal_resource', how='left')
df_wk = pd.merge(df_res,df_ev,on='ParentGlobalID', how='left')

df_wk.columns

Index(['ObjectID', 'GlobalID', '_focal_resource_label', 'sg_dependence',
       'resource_quality_change', 'resource_quantity_change', 'management',
       'compliance', 'ParentGlobalID', 'CreationDate_x', 'Creator_x',
       'EditDate_x', 'Editor_x', 'management_other', 'focal_resource',
       'objectid', 'id', 'date', 'ccp', 'seagrass_quality_change',
       'sg_quality_change_note', 'seagrass_quantity_change',
       'sg_quantity_change_note', 'causes_seagrass_change', 'sg_causes_coded',
       'focal_resources', 'sg_dependence_note', 'resource_quality_change_note',
       'resource_quantity_change_note', 'management_note', 'compliance_note',
       'resource_change_note', 'resource_change_note.1', 'note', 'review_flag',
       'CreationDate_y', 'Creator_y', 'EditDate_y', 'Editor_y', 'x', 'y'],
      dtype='object')

In [38]:
df_wk[['_focal_resource_label', 'sg_dependence',
       'resource_quality_change', 'resource_quantity_change', 'management',
       'compliance', 'ParentGlobalID','management_other', 'focal_resource','id', 'date', 'ccp', 'seagrass_quality_change',
       'sg_quality_change_note', 'seagrass_quantity_change',
       'sg_quantity_change_note', 'causes_seagrass_change', 'sg_causes_coded',
       'focal_resources', 'sg_dependence_note', 'resource_quality_change_note',
       'resource_quantity_change_note', 'management_note', 'compliance_note',
       'resource_change_note', 'resource_change_note.1', 'note', 'review_flag']].to_csv('resource_eco_sens_clean.csv')

In [28]:
# create a dictionary to replace descriptive scale values with interger scale values
sg_dep_dic = {'none':0,'low':1,'mod':2,'high':3}

# function to pull item from dict
def sg_scale(dep):
    scale=sg_dep_dic[dep]
    return scale

# apply function
df_wk['sg_dep_int']=df_wk.apply(lambda row: sg_scale(row['sg_dependence']), axis=1 )

In [29]:
# get the median integer score
df_wk_sg_dep = df_wk.groupby(['focal_resource']).agg(sg_dep_median=('sg_dep_int', np.median), mode = ('sg_dep_int', pd.Series.mode)).reset_index() 
df_wk_sg_dep

,focal_resource,sg_dep_median,mode
0,01,0.0,0
1,02,0.0,0
2,03,0.5,0
3,04,3.0,3
4,06,2.5,3
5,07,0.0,0
6,10,0.0,0
7,11,0.0,0
8,37,0.0,0
9,50,3.0,3


In [30]:
def rescale(min0,max0,min1,max1, x):
    y = (min1-max1)*(x-min0)/(min0-max0) + min1
    return y

df_wk_sg_dep['sg_dep_rescaled']=df_wk_sg_dep.apply(lambda row: rescale(0,3,0,2,row['sg_dep_median']), axis=1 )


In [31]:
def reclass(sg):
    if sg == 0:
        return 0
    elif sg > 1.5:
        return 2
    else: 
        return 1
    
df_wk_sg_dep['sg_dep_reclass'] = df_wk_sg_dep.apply(lambda row: reclass(row['sg_dep_rescaled']), axis=1 )
df_wk_sg_dep


,focal_resource,sg_dep_median,mode,sg_dep_rescaled,sg_dep_reclass
0,01,0.0,0,0.000000,0
1,02,0.0,0,0.000000,0
2,03,0.5,0,0.333333,1
3,04,3.0,3,2.000000,2
4,06,2.5,3,1.666667,2
5,07,0.0,0,0.000000,0
6,10,0.0,0,0.000000,0
7,11,0.0,0,0.000000,0
8,37,0.0,0,0.000000,0
9,50,3.0,3,2.000000,2


In [32]:
df_local_sg_dep = pd.merge(df_wk_sg_dep,species,on='focal_resource', how='left')
df_local_sg_dep[['focal_resource', 'scientific','sg_dep_reclass']].to_csv('local_seagrass_association_scores.csv')



In [33]:
# function to get key
def get_key(val):
    for key, value in sg_dep_dic.items():
        if val == value:
            return key
    return "key doesn't exist"

def cat_dep(sg_dep):
    dep = round(sg_dep)
    cat = get_key(dep)
    return cat

df_wk_sg_dep['sg_dep_cat'] = df_wk_sg_dep.apply(lambda row: cat_dep(row['sg_dep_median']), axis=1 )
df_wk_sg_dep

,focal_resource,sg_dep_median,mode,sg_dep_rescaled,sg_dep_reclass,sg_dep_cat
0,01,0.0,0,0.000000,0,none
1,02,0.0,0,0.000000,0,none
2,03,0.5,0,0.333333,1,none
3,04,3.0,3,2.000000,2,high
4,06,2.5,3,1.666667,2,mod
5,07,0.0,0,0.000000,0,none
6,10,0.0,0,0.000000,0,none
7,11,0.0,0,0.000000,0,none
8,37,0.0,0,0.000000,0,none
9,50,3.0,3,2.000000,2,high


In [34]:
df_wk_sg_dep.to_csv("sg_dep.csv")